Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Installation and configuration
This notebook configures the notebooks in this tutorial to connect to an Azure Machine Learning (AML) Workspace.  You can use an existing workspace or create a new one.

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore
from dotenv import set_key, get_key, find_dotenv
from pathlib import Path
from AIHelpers.utilities import get_auth
from MetricsUtils.storageutils import storageConnection

## Prerequisites

If you have already completed the prerequisites and selected the correct Kernel for this notebook, the AML Python SDK is already installed. Let's check the AML SDK version.

In [ ]:
print("AML SDK Version:", azureml.core.VERSION)

## Set up your Azure Machine Learning workspace

To create or access an Azure ML Workspace, you will need the following information:

* Your subscription id
* A resource group name
* A name for your workspace
* A region for your workspace

**Note**: As with other Azure services, there are limits on certain resources like cluster size associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

If you have a workspace created already, you need to get your subscription and workspace information. You can find the values for those by visiting your workspace in the [Azure portal](http://portal.azure.com). If you don't have a workspace, the create workspace command in the next section will create a resource group and a workspace using the names you provide.

Replace the values in the following cell with your information. If you would like to use service principal authentication as described [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb) make sure you provide the optional values as well. 

In [ ]:
# Azure resources
subscription_id = "{{cookiecutter.subscription_id}}"
resource_group = "{{cookiecutter.resource_group}}"  
workspace_name = "{{cookiecutter.workspace_name}}"  
workspace_region = "{{cookiecutter.workspace_region}}"
image_name = (
    "{{cookiecutter.image_name}}"
)  # e.g. image_name = "{{cookiecutter.image_name}} (avoid underscore in names)"

sql_datastore_name=""
sql_server_name=""     # Name of Azure SQL server
sql_database_name=""   # Name of Azure SQL database
sql_username=""        # The username of the database user to access the database.
sql_password=""        # The password of the database user to access the database.

blob_datastore_name="" # Name of the Datastore  to workspace
datastore_rg = ""
container_name=""      # Name of Azure blob container
account_name=""        # Storage account name
account_key=""         # Storage account key

username = "YOUR_SERVICE_PRINCIPAL_APPLICATION_ID" # Optional for service principal authentication
password = "YOUR_SERVICE_PRINCIPAL_PASSWORD" # Optional for service principal authentication
storageConnString = "YOUR_STORAGE_CONNECTION_STRING"

Create and initialize a dotenv file for storing parameters used in multiple notebooks.

In [ ]:
env_path = find_dotenv()
if env_path == "":
    Path(".env").touch()
    env_path = find_dotenv()

In [ ]:
sql_datastore_name = "ado_sql_datastore" # Hard Code this for this example
blob_datastore_name="ado_blob_datastore"

set_key(env_path, "subscription_id", subscription_id) # Replace YOUR_AZURE_SUBSCRIPTION
set_key(env_path, "resource_group", resource_group)
set_key(env_path, "workspace_name", workspace_name)
set_key(env_path, "workspace_region", workspace_region)
set_key(env_path, "image_name", image_name)

set_key(env_path, "ado_datastore", sql_datastore_name)

set_key(env_path, "username", username)
set_key(env_path, "password", password)

#### Create the workspace
This cell will create an AML workspace for you in a subscription, provided you have the correct permissions.

This will fail when:
1. You do not have permission to create a workspace in the resource group
2. You do not have permission to create a resource group if it's non-existing.
2. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources. If this cell succeeds, you're done configuring AML!  

In [ ]:
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group,
    location=workspace_region,
    create_resource_group=True,
    auth=get_auth(env_path),
    exist_ok=True,
)

Let's check the details of the workspace.

In [ ]:
ws.get_details()

In [ ]:
blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key,
                                                         resource_group=datastore_rg)

In [ ]:
try:
    sql_datastore = Datastore.register_azure_sql_database(workspace=ws,
                                                          datastore_name=sql_datastore_name,
                                                          server_name=sql_server_name,
                                                          database_name=sql_database_name,
                                                          username=sql_username,
                                                          password=sql_password)
except Exception:
    print("failed to sync with SQL DB")

Let's write the workspace configuration for the rest of the notebooks to connect to the workspace.

In [ ]:
ws.write_config()

You are now ready to move on to the [data preperation](01_DataPrep.ipynb) notebook.